# Dropping

In [12]:
import os
import subprocess
import time
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.node import Switch
from mininet.cli import CLI
from mininet.log import setLogLevel, info

class P4Switch(Switch):
    def __init__(self, name, json_file, thrift_port=9090, **params):
        Switch.__init__(self, name, **params)
        self.json_file = json_file
        self.thrift_port = thrift_port
        self.interfaces = []  # Store interfaces to bind

    def start(self, controllers):
        # Get interfaces from Mininet (e.g., s1-eth0, s1-eth1)
        intf_args = " ".join(f"--interface {idx}@{intf.name}" for idx, intf in enumerate(self.intfList()) if intf.name != "lo")
        cmd = f"simple_switch --thrift-port {self.thrift_port} --log-console --log-level trace {intf_args} {self.json_file} &"
        info(f"*** Starting P4 switch {self.name}: {cmd}\n")
        os.system(cmd)
        time.sleep(5)  # Wait for switch to bind interfaces

    def stop(self):
        info(f"*** Stopping P4 switch {self.name}\n")
        os.system("pkill simple_switch")

class NetworkTopo(Topo):
    def __init__(self, json_file):
        Topo.__init__(self)
        s1 = self.addSwitch('s1', cls=P4Switch, json_file=json_file, thrift_port=9090)
        h1 = self.addHost('h1', ip='10.0.1.1/24', mac='00:00:00:00:01:01')
        h2 = self.addHost('h2', ip='10.0.1.2/24', mac='00:00:00:00:01:02')
        self.addLink(h1, s1, port1=0, port2=0)
        self.addLink(h2, s1, port1=0, port2=1)

def compile_p4_program():
    print("Compiling P4 program...")
    try:
        subprocess.run(['p4c', '--target', 'bmv2', '--arch', 'v1model', '-o', '.', 'drop.p4'], check=True)
        print("P4 compilation successful")
        time.sleep(5)
    except subprocess.CalledProcessError as e:
        print(f"P4 compilation failed: {e}")
        exit(1)

def setup_network():
    json_file = "drop.json"
    net = Mininet(topo=NetworkTopo(json_file), controller=None)
    try:
        net.start()
        s1 = net.get('s1')
        s1.cmd('simple_switch_CLI --thrift-port 9090 << EOF\n'
               'mc_mgrp_create 1\n'
               'mc_node_create 0 0 1\n'
               'mc_node_associate 1 0\n'
               'EOF')
        s1.cmd('simple_switch_CLI --thrift-port 9090 << EOF\n'
               'table_add ipv4_filter drop 10.0.1.1 10.0.1.2 =>\n'    # Drop h1 to h2
               'table_add ipv4_filter forward 10.0.1.2 10.0.1.1 => 0\n'  # h2 to h1
               'table_add ipv4_filter forward 10.0.1.1 10.0.1.1 => 0\n'  # h1 to itself
               'table_add ipv4_filter forward 10.0.1.2 10.0.1.2 => 1\n'  # h2 to itself
               'EOF')
        h1, h2 = net.get('h1', 'h2')
        h1.cmd('ifconfig h1-eth0 10.0.1.1/24 up')
        h2.cmd('ifconfig h2-eth0 10.0.1.2/24 up')
        h1.cmd('arp -i h1-eth0 -s 10.0.1.2 00:00:00:00:01:02')
        h2.cmd('arp -i h2-eth0 -s 10.0.1.1 00:00:00:00:01:01')
        h1.cmd('ip route add 10.0.1.0/24 dev h1-eth0')
        h2.cmd('ip route add 10.0.1.0/24 dev h2-eth0')
        
        info("*** h1 interface config:\n")
        print(h1.cmd('ifconfig h1-eth0'))
        info("*** h1 ARP table:\n")
        print(h1.cmd('arp -n'))
        info("*** h2 interface config:\n")
        print(h2.cmd('ifconfig h2-eth0'))
        
        info("*** Starting tcpdump on h2-eth0:\n")
        h2.cmd('tcpdump -i h2-eth0 -w h2_capture.pcap &')
        time.sleep(1)
        
        info("*** Pinging from h1 to h2 (should fail):\n")
        print(h1.cmd('ping -c 10 10.0.1.2'))
        
        info("*** Pinging from h2 to h1 (should succeed):\n")
        print(h2.cmd('ping -c 10 10.0.1.1'))
        
        h2.cmd('pkill tcpdump')
        info("*** tcpdump output (check h2_capture.pcap manually):\n")
        
        info("*** Pinging h1 to h2 with net.ping (mixed results expected):\n")
        print(net.ping([h1, h2]))
        
        CLI(net)
    finally:
        net.stop()

def main():
    setLogLevel('info')
    os.system('sudo pkill -f simple_switch')
    for tool in ['p4c', 'simple_switch', 'simple_switch_CLI']:
        if subprocess.run(['which', tool], capture_output=True).returncode != 0:
            print(f"ERROR: {tool} not found in PATH!")
            exit(1)
    compile_p4_program()
    setup_network()

if __name__ == '__main__':
    main()

Compiling P4 program...
P4 compilation successful


*** Creating network
*** Adding hosts:
h1 h2 
*** Adding switches:
s1 
*** Adding links:
(h1, s1) (h2, s1) 
*** Configuring hosts
h1 h2 
*** Starting controller

*** Starting 1 switches
s1 *** Starting P4 switch s1: simple_switch --thrift-port 9090 --log-console --log-level trace --interface 0@s1-eth0 --interface 1@s1-eth1 forward.json &


Calling target program-options parser
[20:59:55.086] [bmv2] [D] [thread 44106] Set default default entry for table 'MyIngress.ipv4_lpm': MyIngress.drop - 
[20:59:55.086] [bmv2] [D] [thread 44106] Set default default entry for table 'MyIngress.ethernet_exact': MyIngress.broadcast - 
Adding interface s1-eth0 as port 0
[20:59:55.086] [bmv2] [D] [thread 44106] Adding interface s1-eth0 as port 0
Adding interface s1-eth1 as port 1
[20:59:55.095] [bmv2] [D] [thread 44106] Adding interface s1-eth1 as port 1
[20:59:55.105] [bmv2] [I] [thread 44106] Starting Thrift server on port 9090
[20:59:55.105] [bmv2] [I] [thread 44106] Thrift server was started
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Processing packet received on port 0
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Parser 'parser': start
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Parser 'parser' entering state 'start'
[20:59:55.318] [bmv2] [D] [thread 44112] [0.0] [cxt 0] Extracting header 'ethernet'
[20


*** h1 interface config:
*** h1 ARP table:
*** h2 interface config:
*** Starting tcpdump on h2-eth0:


[21:00:00.140] [bmv2] [T] [thread 44120] bm_get_config
[21:00:00.142] [bmv2] [T] [thread 44120] bm_mc_mgrp_create
[21:00:00.142] [bmv2] [D] [thread 44120] mgrp node created for mgid 1
[21:00:00.142] [bmv2] [T] [thread 44120] bm_mc_node_create
[21:00:00.142] [bmv2] [D] [thread 44120] node created for rid 0
[21:00:00.142] [bmv2] [T] [thread 44120] bm_mc_node_associate
[21:00:00.142] [bmv2] [D] [thread 44120] node associated with mgid 1
[21:00:00.205] [bmv2] [T] [thread 44122] bm_get_config
[21:00:00.206] [bmv2] [T] [thread 44122] bm_table_add_entry
[21:00:00.206] [bmv2] [D] [thread 44122] Entry 0 added to table 'MyIngress.ipv4_lpm'
[21:00:00.206] [bmv2] [D] [thread 44122] Dumping entry 0
Match key:
* hdr.ipv4.dstAddr    : LPM       0a000101/32
Action entry: MyIngress.forward - 0,

[21:00:00.207] [bmv2] [T] [thread 44122] bm_table_add_entry
[21:00:00.207] [bmv2] [D] [thread 44122] Entry 1 added to table 'MyIngress.ipv4_lpm'
[21:00:00.207] [bmv2] [D] [thread 44122] Dumping entry 1
Match ke

...

[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Processing packet received on port 0
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser 'parser': start
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser 'parser' entering state 'start'
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Extracting header 'ethernet'
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser state 'start': key is 86dd
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] Bytes parsed: 14
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Parser 'parser': end
[21:00:01.150] [bmv2] [D] [thread 44112] [19.0] [cxt 0] Pipeline 'ingress': start
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] forward.p4(106) Condition "hdr.ipv4.isValid()" (node_2) is false
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] forward.p4(108) Condition "hdr.ethernet.isValid()" (node_4) is true
[21:00:01.150] [bmv2] [T] [thread 44112] [19.0] [cxt 0] Applying table 'MyIngress

*** tcpdump output (check h2_capture.pcap manually):
*** Pinging h1 to h2 with net.ping:
h1 -> h2 
h2 -> h1 
*** Results: 0% dropped (2/2 received)
*** Starting CLI:


[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Processing packet received on port 0
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser': start
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser' entering state 'start'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Extracting header 'ethernet'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser state 'start': key is 0800
[21:00:10.430] [bmv2] [T] [thread 44112] [44.0] [cxt 0] Bytes parsed: 14
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser' entering state 'parse_ipv4'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Extracting header 'ipv4'
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser state 'parse_ipv4' has no switch, going to default next state
[21:00:10.430] [bmv2] [T] [thread 44112] [44.0] [cxt 0] Bytes parsed: 34
[21:00:10.430] [bmv2] [D] [thread 44112] [44.0] [cxt 0] Parser 'parser': end
[21:00:10.430] [bmv2] [D] [th

mininet>  exit


*** Stopping 0 controllers

*** Stopping 2 links
..
*** Stopping 1 switches
s1 *** Stopping P4 switch s1

*** Stopping 2 hosts
h1 h2 
*** Done
